# Stokes Gleichung (2D)

## Fallender Block (konstante Viskosität)

Beschäftigen wir uns zu erst mit einem instantanen, d.h. einem zeit-unabhängigen, und isoviskosen,d.h. konstanter Viskosität, Problem, dem **fallenden Block**. Der fallende Block ist ein relative einfach aufzusetztendes Problem und ist sehr hilfreich, um mit dem bestehenden Code einem Benchmark durchzuführen. 

### Problem

Anders als im vollständigen Benchmark für dieses Problem, konzentrieren wir uns hier nur auf den **isoviskosen** Fall (im Benchmark betrachtet man die Sinkgeschwindigkeit und Deformation des Blockes in Abhängigkeit des Viskositätskontrastes [siehe hier](../../examples/Benchmarks/)). 

Dabei nehmen wir einen quadratischen Körper mit einer gewissen Breite (**W**), Höhe (**T**), und Dichte ($\rho_b$) an, der sich in einem bestimmten viskosen Medium ($\eta_m$) mit einer gewissen Dichte ($\rho_m$) befindet. Die zu modellierende Umgebung ist ebenfals quadratisch mit einer gewissen Länge ($L=500\ \textrm{km}$) und Höhe ($H=500\ \textrm{km}$) und wir nehmen überall *free slip* Geschwindigkeitsrandbedingungen an.  

<img src="../Figures/Exercise09_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau 

Laden wir zuerst einmal die noptwendigen Module zur Visualisierung der Ergebnisse, Lösen des linearen Gleichungssystems, Aufstellen der Anfangsbedingungen, und dem Lösen der zweidimensionalen Impulserhaltuing. 

Eine detaillierte Beschreibung wie die Modelldomäne durch ein numerisches Gitter aufgeteilt werden kann, sowie die Diskretisierung der **Impulserhaltung** und **Massenerhaltung** in Abhängigkeit von *free slip* oder *no slip* Randbedingungen ist [hier](../../examples/StokesEquation/2D/README.md) gegeben. Auf Grund des Umfanges verzichten wir hier auf eine detaillierte Erklärung. 

In [ ]:
using Plots
using ExtendableSparse
using GeoModBox.InitialCondition, GeoModBox.MomentumEquation.TwoD

Mit Hilfe der eingebauten Funktion ```IniPhase()``` lässt sich die Anfangsdichteverteilung für unser Problem *block* erstellen. Je nach Bedarf, erstellt die Funktion eine Anfangsverteilung von bestimmten Größen auf dem numerischen Gitter oder auf den Markern. Zur Nutzung der Funktion brauchen wir das Tuple ```Ini```, in dem die Verteilung definiert ist (hier für die Phase *p*).

In [ ]:
# Define Initial Condition ========================================== #
# Density --- 
#   1) block
Ini         =   (p=:block,) 
# ------------------------------------------------------------------- #
# Plot Settings ===================================================== #
Pl  =   (
    qinc    =   5,
    qsc     =   100*(60*60*24*365.25)*1e2
)
# ------------------------------------------------------------------- #

Als nächstes definieren wir die Geometrie unsere Modeldomäne. 

In [ ]:
# Geometry ========================================================== #
M       =   (
    xmin    =   0.0,
    xmax    =   500.0e3,    # [ m ]
    ymin    =   -500.0e3,   # [ m ]
    ymax    =   0.0,
)
# -------------------------------------------------------------------- #

Nun definieren wir unsere Gitter Auflösung $\left(nc_x = nc_y = 50\right)$ und das numerische Gitter.

Dabei müssen wir sowohl die Gitterbreite $\Delta{x}$ und $\Delta{y}$ als auch die Koordinaten der unterschiedlichen Gitter (*Eckpunkte* und *Zentroids*) angeben. Dazu definieren wir zuerst die jeweiligen 1-D Koordinatenvektoren, aus denen dann die 2-D Koordinatengitter erstellt werden.  

In [ ]:
# Grid =============================================================== #
NC      =   (
    x   =   50, 
    y   =   50,
)
NV      =   (
    x   =   NC.x + 1,
    y   =   NC.y + 1,
)
Δ       =   (
    x   =   (M.xmax - M.xmin)/NC.x,
    y   =   (M.ymax - M.ymin)/NC.y,
)
x       =   (
    c   =   LinRange(M.xmin+Δ.x/2,M.xmax-Δ.x/2,NC.x),
    ce  =   LinRange(M.xmin - Δ.x/2.0, M.xmax + Δ.x/2.0, NC.x+2),
    v   =   LinRange(M.xmin,M.xmax,NV.x),
)
y       =   (
    c   =   LinRange(M.ymin+Δ.y/2,M.ymax-Δ.y/2,NC.y),
    ce  =   LinRange(M.ymin - Δ.x/2.0, M.ymax + Δ.x/2.0, NC.y+2),
    v   =   LinRange(M.ymin,M.ymax,NV.y),
)
x1      =   (
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# -------------------------------------------------------------------- #

Nun müssen wir die physikalischen Parameter definieren, die Graviationsbeschleunigung $g$, die Viskosität $\eta_0$ (10<sup>21</sup> Pa s), die Hintergunddichte $\rho_0$ (3200 kg/m<sup>3</sup>), und die Dichte des Blocks $\rho_1$ (3300 kg/m<sup>3</sup>).

Der Vektor ```phase``` beinhaltet die Phasennummer, welche in der Funktion ```IniPhase()``` verwendet wird um den zugehörigen Punkten (Gitter oder Marker) die jeweilge Phase zuzuordnen. Der Vektor $\rho$ beinhaltet die Dichtewerte die der jeweilige Phase zugeordnet sind. 

In [ ]:
# Physics ============================================================ #
g       =   9.81

η₀      =   1.0e21

ρ₀      =   3200.0          #   Background density
ρ₁      =   3300.0          #   Block density
ρ       =   [ρ₀,ρ₁] 

phase   =   [0,1]
# ------------------------------------------------------------------- #

Nun legen wir die Größen der benötigten Felder fest. 

In [ ]:
# Allocation ======================================================== #
D   =   (
    vx      =   zeros(Float64,NV.x,NC.y+2),
    vy      =   zeros(Float64,NC.x+2,NV.y),
    Pt      =   zeros(Float64,NC...),
    p       =   zeros(Int64,NC...),
    p_ex    =   zeros(Int64,NC.x+2,NC.y+2),
    ρ       =   zeros(Float64,NC...),
    vxc     =   zeros(Float64,NC...),
    vyc     =   zeros(Float64,NC...),
    vc      =   zeros(Float64,NC...),
)
# ------------------------------------------------------------------- #

Jetz lassen sich die Randbedingungen und die Anfangsbedingungen festlegen. 

Für die Randbedingungen benutzen wir wieder das Tuple ```VBC```, in dem die Art des Randes (*free slip* oder *no slip*) an der jeweiligen Position (**E**, **W**, **S**,**N**) definiert ist. Der angegeneben Wert ```val``` bestimmt dann den Wert der Geschwindigkeit an dem jeweiligen Rand. 

Was bedeutet das für die *free slip* und *no slip* Bedingungen?

In [ ]:
# Boundary Conditions =============================================== #
VBC     =   (
    type    =   (E=:freeslip,W=:freeslip,S=:freeslip,N=:freeslip),
    # type    =   (E=:noslip,W=:noslip,S=:noslip,N=:noslip),
    val     =   (E=zeros(NV.y),W=zeros(NV.y),S=zeros(NV.x),N=zeros(NV.x)),
)
# ------------------------------------------------------------------- #

Zum Aufstellen der Anfangsbedingungen, verwenden wir die Funktion ```IniPhase``` in der dem jeweiligen Punkt die dazugehörige Phase zugeteilt wird. Darauf weisen wir der jeweilgen Phase den entsprechenden Dichte Wert hinzu. 

In [ ]:
# Initial Condition ================================================= #
IniPhase!(Ini.p,D,M,x,y,NC;phase)
for i in eachindex(phase)
    D.ρ[D.p.==phase[i]] .= ρ[i]
end
# ------------------------------------------------------------------- #

Nun müssen die Parameter für das lineare Gleichngssstem erstellt werden. 

Die Nummerierung der Gleichungen für die *x-Komponente* und die *y-Komnponente* der Impulserhaltung, sowie die der *Massenerhaltung*, als auch die Diskretisierung der zu lösenden Gleichungen mit Hilfe der finiten Differenzen Methode ist im Detail [hier](../../examples/StokesEquation/2D/README.md) beschrieben. 

Außerdem muss noch Lösungsvektor ```χ``` initialisiert werden. Die Größe von beiden ist durch die maximale Anzahl der Gleichungen gegeben. 

In [ ]:
# System of Equations =============================================== #
# Numbering, without ghost nodes! ---
off    = [  NV.x*NC.y,                          # vx
            NV.x*NC.y + NC.x*NV.y,              # vy
            NV.x*NC.y + NC.x*NV.y + NC.x*NC.y]  # Pt

Num    =    (
    Vx  =   reshape(1:NV.x*NC.y, NV.x, NC.y), 
    Vy  =   reshape(off[1]+1:off[1]+NC.x*NV.y, NC.x, NV.y), 
    Pt  =   reshape(off[2]+1:off[2]+NC.x*NC.y,NC...),
)
χ       =   zeros(maximum(Num.Pt))
# ------------------------------------------------------------------- #

Mit Hilfe der Funktion ```Assemblyc()``` lassen sich die Koeffizienten für die Koeffizientenmatrix $\bold{K}$ zuweisen. Die Funktion weist der jeweiligen Gleichung den Wert der benötigten Koeffizienten zu. 

In [ ]:
# Assemble Coefficients ============================================= #
K       =   Assemblyc(NC, NV, Δ, η₀, VBC, Num)
# ------------------------------------------------------------------- #

Nun müssen wir den rechte-Seite Vektor noch mit der Information der Anfangsbedingungen und der Randbedingungen aktualisieren mit Hilfe der Funkition ```updaterhsc()```. Die Funktion weist der jeweiligen Gleichung den Wert der jeweiligen rechten Seite des linearen Gleichungssystems hinzu. 

In [ ]:
# Update RHS ======================================================== #
rhs     =   updaterhsc( NC, NV, Δ, η₀, D.ρ, -g, VBC, Num )
# ------------------------------------------------------------------- #

Nun lässt sich das Problem mit Hilfe einer Rechtsdivision lösen. 

In [ ]:
# Solve System of Equations ========================================= #
χ       =   K \ rhs
# ------------------------------------------------------------------- #

Anschließend müssen die neuen Werte für die horizontale und vertikale Geschwindikeit, sowie der Druck dem dazugehörigen Gitter hinzugewiesen werden. 

In [ ]:
# Update Unknown Variables ========================================== #
D.vx[:,2:end-1]     .=  χ[Num.Vx]
D.vy[2:end-1,:]     .=  χ[Num.Vy]
D.Pt                .=  χ[Num.Pt]
# ------------------------------------------------------------------- #

Zur besseren Visualiesierung berechnen wir nun noch die Geschwindigkeiten auf den *Zentroids*. 

In [ ]:
# Get the velocity on the centroids ---
for i = 1:NC.x
    for j = 1:NC.y
        D.vxc[i,j]  = (D.vx[i,j+1] + D.vx[i+1,j+1])/2
        D.vyc[i,j]  = (D.vy[i+1,j] + D.vy[i+1,j+1])/2
    end
end
@. D.vc        = sqrt(D.vxc^2 + D.vyc^2)

@show(minimum(D.vc))
@show(maximum(D.vc))


Zum Schluss, stellen wir die Dichteverteilung und die Geschwindigkeiten graphisch dar.

In [ ]:
p = heatmap(x.c./1e3,y.c./1e3,D.ρ',color=:inferno,
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="Density",
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3), 
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=1)
quiver!(p,x.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
        y.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
        quiver=(D.vx[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc,
        D.vyc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc), 
        la=0.5,
        color="white",layout=(2,2),subplot=1)
heatmap!(p,x.c./1e3,y.c./1e3,D.vxc',
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="V_x",color=cgrad(:batlow),
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=3)
heatmap!(p,x.c./1e3,y.c./1e3,D.vyc',
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="V_y",color=cgrad(:batlow),
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=4)
heatmap!(p,x.c./1e3,y.c./1e3,D.Pt',
        xlabel="x[km]",ylabel="y[km]",colorbar=false,
        title="P_t",color=cgrad(:lipari),
        aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
        ylims=(M.ymin/1e3, M.ymax/1e3),
        layout=(2,2),subplot=2)
display(p)

savefig(p,string("./Results/09_FallingBlock_Instantan.png"))